## 爬取目标

<div align=center>
    <img src="../../../images/fb0cc800f904f30931b46f0af6dd6c257e0a709d67c0dec84766a0f3562b9be4.png" width=75%>
    </img>
</div>

- 1. 提取每一页的电影列表,以及每个电影的详情页

- 2. 提取每部电影的`名称`、`封面`、`类别`、`上映时间`、`评分`、`剧情简介`

- 3. 存入数据库

- 4. 多进程

## 爬取过程

### 1. 提取 电影名及对应URL 列表（`采取正则表达式`）

#### `(1)导库 以及 获取对应URL的HTML`

In [2]:
import requests
import re
from pyquery import PyQuery as pq

origin_url = 'https://ssr1.scrape.center'

## 给定 url 获取该 url 对应的 html
def get_html(url):
    response = requests.get(url)
    html = response.text
    return html

#### `(2)获取所有页的 页URL`

In [3]:
## 获取每一页的 url
def get_all_page_url(html):
    
    movies_page = re.findall('<li.*?number.*?>.*?<a href="(.*?)">(.*?)</a>.*?</li>', html, re.S)
    
    page_infos = []
    page_url_list = []
    for page in movies_page:
        page_info = {
            'page_name':page[1],
            'page_url':'https://ssr1.scrape.center' + page[0]
        }
        page_infos.append(page_info)
        page_url_list.append('https://ssr1.scrape.center' + page[0])
    
    return page_url_list

#### `(3)获取某一页中所有的电影及对应URL`

In [4]:
## 获取 1 页的所有电影的名字和对应url
def get_all_movie_name_and_url_from_current_page(current_page_html):
    p = '<div.*?index.*?>.*?<div.*?"p-h el-col el-col-24 el-col-xs-9 el-col-sm-13 el-col-md-16">.*?<a.*?href="(.*?)".*?>.*?<h2.*?>(.*?)</h2>.*?</a>'
    # 一页的电影基本信息（电影名、电影url链接）
    page_movies = re.findall(p, current_page_html, re.S)

    current_page_movies_list = []
    for movie in page_movies:
        movie_in_page = {
            'name':movie[1],
            'url':'https://ssr1.scrape.center' + movie[0]
        }
        current_page_movies_list.append(movie_in_page)
    return current_page_movies_list

# get_all_movie_name_and_url_from_current_page(None)

#### `(4)代码合并, 获取所有的电影URL及名字`

In [5]:
'''
    获取 -> 所有页中 -> 所有的电影名+电影url
'''

# 获取 origin 页的html
origin_html = get_html(origin_url)

# 获取所有页的url
all_page_url_list = get_all_page_url(origin_html)

# 获取所有电影名和URL
i = 0
all_movie_name_and_url_list = []
for current_page_url in all_page_url_list:
    i += 1
    '''
        加一个判错机制
    '''
    try:
        # print('开始采集第{}页'.format(i))
        # print('第{}页URL:{}'.format(i,current_page_url))
        # 获取每一页的html
        current_page_html = get_html(current_page_url)
        # 获取每一页中所有电影的名字和URL
        current_page_movies_list = get_all_movie_name_and_url_from_current_page(current_page_html)
        # 将各页的（电影名、电影URL）合并起来
        all_movie_name_and_url_list += current_page_movies_list
        print('第{}页采集完成\n'.format(i))
    except:
        print('第{}页报错\n'.format(i))
        continue

all_movie_name_and_url_list[:5]

第1页采集完成

第2页采集完成

第3页采集完成

第4页采集完成

第5页采集完成

第6页采集完成

第7页采集完成

第8页采集完成

第9页采集完成

第10页采集完成

第11页采集完成



[{'name': '霸王别姬 - Farewell My Concubine',
  'url': 'https://ssr1.scrape.center/detail/1'},
 {'name': '这个杀手不太冷 - Léon', 'url': 'https://ssr1.scrape.center/detail/2'},
 {'name': '肖申克的救赎 - The Shawshank Redemption',
  'url': 'https://ssr1.scrape.center/detail/3'},
 {'name': '泰坦尼克号 - Titanic', 'url': 'https://ssr1.scrape.center/detail/4'},
 {'name': '罗马假日 - Roman Holiday',
  'url': 'https://ssr1.scrape.center/detail/5'}]

### 采集电影的详细信息

- 电影名（英文名）
- 评分
- 类别
- 上映地点
- 上映时间
- 剧情简介
- 导演
- 演员

In [6]:
def get_movie_detail_info(movie_html):
    # 电影名
    name = re.search('<div.*?p-h el-col el-col-24 el-col-xs-16 el-col-sm-12.*?>.*?<h2.*?>(.*?)</h2>', movie_html, re.S).group(1)

    # 类别
    all_categories_str = re.search('<div.*?categories.*?>.*?(<button.*?/button.*?button.*?/button>).*?</div>', movie_html, re.S).group(1)
    categories_list = re.findall('<span>(.*?)</span>', all_categories_str, re.S)

    # 上映地点 及 片长
    addr_and_time = re.search('<div.*?m-v-sm info">.*?<span.*?>(.*?)</span>.*?<span.*?/span>.*?<span.*?>(.*?)</span>.*?</div>', movie_html, re.S)
    addr = addr_and_time.group(1)
    time = addr_and_time.group(2)

    # 上映时间
    first_date = re.search('<div.*?m-v-sm info">.*?</div>.*?<div.*?m-v-sm info">.*?<span.*?>(.*?)</span>.*?</div>', movie_html, re.S).group(1)

    # 剧情简介
    drama = re.search('<div.*?drama">.*?<p.*?>(.*?)</p>', movie_html, re.S).group(1).strip()

    # 评分
    rate = float(re.search('el-col el-col-24 el-col-xs-8 el-col-sm-4.*?<p.*?>(.*?)</p>', movie_html, re.S).group(1).strip())

    # 导演
    director = re.search('subtitle.*?el-card__body.*?<p.*?>(.*?)</p>', movie_html, re.S).group(1)

    movie_detail_info = {
        'name':name,
        'categories':categories_list,
        'rate':rate,
        'time':time,
        'addr':addr,
        'date':first_date,
        'director':director,
        'drama':drama,
    }

    return movie_detail_info

In [7]:
i = 0
all_movie_detail_info = []
for movie_name_and_url in all_movie_name_and_url_list:  
    # 获取movie_html
    movie_html = get_html(movie_name_and_url['url'])
    
    # 获取 movie_detail_info
    movie_detail_info = get_movie_detail_info(movie_html)
    
    if movie_name_and_url['name'] == movie_detail_info['name']:
        all_movie_detail_info.append(movie_detail_info)
        i += 1
        print('第{}个采集完成'.format(i))
    else:
        i += 1
        print('第{}个报错, 电影信息{}'.format(i, movie_name_and_url))
        continue
print(i)

第1个采集完成
第2个采集完成
第3个采集完成
第4个采集完成
第5个采集完成
第6个采集完成
第7个采集完成
第8个采集完成
第9个采集完成
第10个采集完成
第11个采集完成
第12个采集完成
第13个采集完成
第14个采集完成
第15个采集完成
第16个采集完成
第17个采集完成
第18个采集完成
第19个采集完成
第20个采集完成
第21个采集完成
第22个采集完成
第23个采集完成
第24个采集完成
第25个采集完成
第26个采集完成
第27个采集完成
第28个采集完成
第29个采集完成
第30个采集完成
第31个采集完成
第32个采集完成
第33个采集完成
第34个采集完成
第35个采集完成
第36个采集完成
第37个采集完成
第38个采集完成
第39个采集完成
第40个采集完成
第41个采集完成
第42个采集完成
第43个采集完成
第44个采集完成
第45个采集完成
第46个采集完成
第47个采集完成
第48个采集完成
第49个采集完成
第50个采集完成
第51个采集完成
第52个采集完成
第53个采集完成
第54个采集完成
第55个采集完成
第56个采集完成
第57个采集完成
第58个采集完成
第59个采集完成
第60个采集完成
第61个采集完成
第62个采集完成
第63个采集完成
第64个采集完成
第65个采集完成
第66个采集完成
第67个采集完成
第68个采集完成
第69个采集完成
第70个采集完成
第71个采集完成
第72个采集完成
第73个采集完成
第74个采集完成
第75个采集完成
第76个采集完成
第77个采集完成
第78个采集完成
第79个采集完成
第80个采集完成
第81个采集完成
第82个采集完成
第83个采集完成
第84个采集完成
第85个采集完成
第86个采集完成
第87个采集完成
第88个采集完成
第89个采集完成
第90个采集完成
第91个采集完成
第92个采集完成
第93个采集完成
第94个采集完成
第95个采集完成
第96个采集完成
第97个采集完成
第98个采集完成
第99个采集完成
第100个采集完成
100


In [8]:
len(all_movie_detail_info)
all_movie_detail_info[0:2]

[{'name': '霸王别姬 - Farewell My Concubine',
  'categories': ['剧情', '爱情'],
  'rate': 9.5,
  'time': '171 分钟',
  'addr': '中国内地、中国香港',
  'date': '1993-07-26 上映',
  'director': '陈凯歌',
  'drama': '影片借一出《霸王别姬》的京戏，牵扯出三个人之间一段随时代风云变幻的爱恨情仇。段小楼（张丰毅 饰）与程蝶衣（张国荣 饰）是一对打小一起长大的师兄弟，两人一个演生，一个饰旦，一向配合天衣无缝，尤其一出《霸王别姬》，更是誉满京城，为此，两人约定合演一辈子《霸王别姬》。但两人对戏剧与人生关系的理解有本质不同，段小楼深知戏非人生，程蝶衣则是人戏不分。段小楼在认为该成家立业之时迎娶了名妓菊仙（巩俐 饰），致使程蝶衣认定菊仙是可耻的第三者，使段小楼做了叛徒，自此，三人围绕一出《霸王别姬》生出的爱恨情仇战开始随着时代风云的变迁不断升级，终酿成悲剧。'},
 {'name': '这个杀手不太冷 - Léon',
  'categories': ['剧情', '动作'],
  'rate': 9.5,
  'time': '110 分钟',
  'addr': '法国',
  'date': '1994-09-14 上映',
  'director': '吕克·贝松',
  'drama': '里昂（让·雷诺 饰）是名孤独的职业杀手，受人雇佣。一天，邻居家小姑娘马蒂尔德（纳塔丽·波特曼 饰）敲开他的房门，要求在他那里暂避杀身之祸。原来邻居家的主人是警方缉毒组的眼线，只因贪污了一小包毒品而遭恶警（加里·奥德曼 饰）杀害全家的惩罚。马蒂尔德 得到里昂的留救，幸免于难，并留在里昂那里。里昂教小女孩使枪，她教里昂法文，两人关系日趋亲密，相处融洽。 女孩想着去报仇，反倒被抓，里昂及时赶到，将女孩救回。混杂着哀怨情仇的正邪之战渐次升级，更大的冲突在所难免……'}]

# 数据存储（MongoDB）

#### ***`a.连接数据库`***

In [10]:
import pymongo

client = pymongo.MongoClient('localhost', port=27017)
database = client.test
collection = database.douban_top

#### ***`b.插入数据`***

In [17]:
result = collection.insert_many(all_movie_detail_info[5:])

#### ***`c.结果`***

<div align=center>
<img src="../../../images/e9bb957df16a5367abf0ab3a78a3688e7c699cd0d037f378a71a0daa6348139a.png" width="75%"></img></div>

# TODO

- 迫切
  - ❌`categories`字段
    - 电影信息的 `categories` 字段只能获取其中的两个类别，如果电影的类别有3个或者更多个，第三个及之后的类别都会被排除掉

- 美化
  - ❌采集进度条